# Testing KadanoffBaym.jl

In [ ]:
include("../src/KadanoffBaym.jl")
using OrdinaryDiffEq # for ODEProblem and solve
using RecursiveArrayTools # For ArrayPartition
using PyCall
qt = pyimport("qutip")
np = pyimport("numpy")
using PyPlot

PyPlot.matplotlib.rc("text", usetex=true)
PyPlot.matplotlib.rc("font", family="serif", size=16)

## Model

$$
    H = J \left( a_1^\dagger a_2^\phantom{\dagger} + \mathrm{h.c.} \right)
$$

## Solving

In [ ]:
# time parameters
T = 1.0
dt = 0.01 # this algorithm requires a fixed time
n = Int(1/dt) 

# quantum numbers
dim = 2

# Define your Green functions at (t0, t0), time-arguments at the beginning
Lesser0 = KadanoffBaym.LesserGF(zeros(ComplexF64, 1, 1, dim, dim))
Greater0 = KadanoffBaym.GreaterGF(zeros(ComplexF64, 1, 1, dim, dim))

# Hamiltonian
J = 2*pi*1.0/2
h = ComplexF64[0.0 J; J 0.0]

# initial condition
N_0 = 1

Lesser0[1,1,1,1] = -1.0im * N_0
Greater0[1,1,1,1] = -1.0im * (N_0 + 1)
Greater0[1,1,2,2] = -1.0im

# Pack them in an ArrayPartition
u0 = ArrayPartition(Lesser0, Greater0);

# Remember that `u` here is also an ArrayPartition-like element
function f(u, p, t, t′)
    u1, u2 = u.x[1], u.x[2]
    rhs1 = -1.0im * (h * u1[t,t′])
    rhs2 = -1.0im * (h * u2[t,t′])
  return ArrayPartition(rhs1, rhs2)
end

# ODE problem is defined by the rhs, initial condition and time span
prob = ODEProblem(f, u0, (0.0, T))

# Algorithm to timestep is the (Kadanoff-Baym) ABM43 (only really this one exists)
alg = KadanoffBaym.KB{ABM43}()

sol = solve(prob, alg, dt);

In [ ]:
stop = Int(n/2) + 1

times = range(0, length=n + 1, stop=T) |> collect;

# defining the tau times
times_tau = 2 .* vcat([-times[stop - (k - 1)] for k in 1:stop - 1], times[1:stop]);

## QuTiP benchmark

In [ ]:
n_max = 2

psi0_list = [qt.basis(n_max + 1, N_0), qt.basis(n_max + 1, 0)]
psi0 = qt.tensor(psi0_list)

a_1_list = [qt.destroy(n_max + 1), qt.qeye(n_max + 1)]
a_1  = qt.tensor(a_1_list)    
a_2_list = [qt.qeye(n_max + 1), qt.destroy(n_max + 1)]
a_2  = qt.tensor(a_2_list)    

H  = J * a_1.dag() * a_2
H += H.dag();

In [ ]:
observables = [a_1.dag()*a_1, a_2.dag()*a_2];

In [ ]:
me = qt.mesolve(H, psi0, times, [], observables)
t_sols = qt.mesolve(H, psi0, times, []); # t_sols.states returns state vectors

#### Two times

In [ ]:
tau_t_sols = Dict()
for k in 1:length(t_sols.states)
    tau_t_sols[k] = qt.mesolve(H, a_1 * t_sols.states[k] * t_sols.states[k].dag(), times).states
end

a_1_dag_a_1 = zeros(ComplexF64, length(t_sols.states), length(t_sols.states))
for k in 1:length(t_sols.states)
    for l in 1:length(t_sols.states)
        a_1_dag_a_1[k, l] = (a_1.dag() * tau_t_sols[k][l]).tr()    
    end
end

In [ ]:
unskewed_a_1_dag_a_1 = zeros(ComplexF64, length(t_sols.states), 2*length(t_sols.states) - 1)
for (k, x) in enumerate([a_1_dag_a_1[k, :] for k in 1:length(t_sols.states)])
    for (l, y) in enumerate(x)
        ind = k + l - 1 # verify the -1 relative to the original python code
        unskewed_a_1_dag_a_1[k, ind] = y  
    end
end

## Plotting

In [ ]:
GL_11 = [-imag(sol.u.x[1][k, k, 1, 1]) for k in 1:n+1]
GL_22 = [-imag(sol.u.x[1][k, k, 2, 2]) for k in 1:n+1];

In [ ]:
# building the tau Green functions
GL_11_tau = [sol.u.x[1][stop - (k - 1), stop + (k - 1), 1, 1] for k in 1:stop]
GL_11_tau_reversed = [GL_11_tau[stop - (k - 1)] for k in 1:stop];
GL_11_tau_full = vcat(-conj(GL_11_tau_reversed[1:end-1]), GL_11_tau);

a_1_dag_a_1_tau = [unskewed_a_1_dag_a_1[stop - (k - 1), stop + (k - 1)] for k in 1:stop]
a_1_dag_a_1_tau_reversed = [a_1_dag_a_1_tau[stop - (k - 1)] for k in 1:stop]
a_1_dag_a_1_tau_full = vcat(conj(a_1_dag_a_1_tau_reversed[1:end-1]), a_1_dag_a_1_tau);

In [ ]:
figure(figsize=(12, 3))
subplot(121)
plot(times, GL_11)
plot(times, me.expect[1], c="k", ls="--")
plot(times, [real(unskewed_a_1_dag_a_1[k, k]) for k in 1:n+1], c="r", ls=":", alpha=0.5)
plot(times, GL_22)
plot(times, me.expect[2], c="k", ls="--")
xlim(0, T)
ylim(0, N_0)
xlabel("\$t\$")

subplot(122)
plot(times_tau, -imag(GL_11_tau_full))
plot(times_tau, a_1_dag_a_1_tau_full, c="k", ls="--")
xlim(-T, T)
xlabel("\$\\tau\$")
# ylim(0, 1.0)

tight_layout()
# savefig("test.pdf")

In [ ]:
figure(figsize=(12, 3))
subplot(121)
plot(times, GL_11 - me.expect[1], c="k", ls="--")
plot(times, GL_22 - me.expect[2], c="k", ls="--")
xlim(0, T)
# ylim(0, N_0)
xlabel("\$t\$")

subplot(122)
plot(times_tau, -imag(GL_11_tau_full) - a_1_dag_a_1_tau_full, c="k", ls="--")
xlim(-T, T)
xlabel("\$\\tau\$")
# ylim(0, 1.0)

tight_layout()